In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


"""Change to the data folder"""
new_path = "../new_train/"
val_path = "../new_val_in/"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None,training=True):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform
        self.training = training

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)

#         varMin =[-5.37872696e+00,  0.00000000e+00, -6.39434204e+01, -8.02092514e+01,
#   0.00000000e+00,  0.00000000e+00,  1.00000000e+06,  1.00000000e+06]
#         varMax = [ 4.72452881e+03,  4.08314209e+03,  7.73842545e+01,  0.00000000e+00,
#   4.73215820e+03,  4.07677856e+03, -1.00000000e+06, -1.00000000e+06]
        columns = ['p_in','p_in','v_in','v_in','lane','lane','lane_norm','lane_norm']
        outColumns = ['p_out','p_out','v_out','v_out']
    #FOR 1000 rows, no shuffle
#     varMin=[ 0.00000000e+00,  0.00000000e+00, -3.83204346e+01, -4.79440918e+01,
#   0.00000000e+00,  0.00000000e+00,  1.00000000e+06,  1.00000000e+06]
#         varMax=[ 4.70824121e+03,  4.04640869e+03,  7.00706635e+01,  0.00000000e+00,
#   4.71727344e+03,  4.07275757e+03, -1.00000000e+06, -1.00000000e+06]
#   FOR ALL ROWS
#         varMaxOutput= [4773.,   4097.7,   193.19,  194.33]
#         varMinOutput=[ -53.912,    0.,    -210.04,  -187.71 ]
#         varMaxInput=[4748.2,   4096.1,    252.32,   183.53,  4791.6,   4121.4,     18.801,   16.702]
#         varMinInput=[ -46.958,    0.,    -222.63,  -179.87,   -75.963,    0.,     -18.564,  -16.691]
#         #Changed to 4 because we don't need lane and lane norm
#         print("here in argo")
#         print("before p_out ",data['p_out'])
#         print("before v_out ",data['v_out'])
#         for i in range(8):
#             j = i % 2
#             data[columns[i]][j] = (data[columns[i]][j] - varMinInput[i]) / (varMaxInput[i] - varMinInput[i])
#             if i < 4 and self.training:
#                 data[outColumns[i]][j] = (data[outColumns[i]][j] - varMinOutput[i]) / (varMaxOutput[i] - varMinOutput[i]) 
#         data['p_out'][0] = (data['p_out'][0] - -5.37872696e+00) / (4.72452881e+03 - -5.37872696e+00)
#         print("after p_out ",data['p_out'])
#         print("after v_out ",data['v_out'])
        return data


train_dataset  = ArgoverseDataset(data_path=new_path)
val_dataset = ArgoverseDataset(data_path=val_path,training=False)
#print((val_dataset[0]))
#print(len(train_dataset[0]))

### Create a loader to enable batch processing

In [3]:
batch_sz = 200
def closest_node(node, nodes):
    deltas = nodes - node
    dist_2 = np.einsum('ij,ij->i', deltas, deltas)
    return np.argmin(dist_2)

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inpTotal = []
    outTotal = []
    inp = []
    out = []
    city = []
#     print("pIn",batch[0]['p_in'])
    numbRows = 60
    for scene in batch:
#         print("scenePin",scene['p_in'])
        cityyy = numpy.zeros((60,1,8))
        #print(cityyy)
        if scene['city'] == 'PIT':
            cityyy[:,:,:] = 1
            #print("mummamia",cityyy)
        city.append(numpy.dstack([cityyy]))
        laneInp = numpy.zeros((numbRows , 19,2))
        lane_norm_inp = numpy.zeros((numbRows , 19,2))
        laneOut = numpy.zeros((numbRows , 30,2))
        lane_norm_out = numpy.zeros((numbRows , 30,2))
        i = 0
#         print(scene['p_in'])
        while scene['car_mask'][i] == 1:
             i+= 1
        for x in range(i):
            for y in range(19):
                index = closest_node(scene['p_in'][x][y],scene['lane'][:,:2])
                laneInp[x][y][0] = scene['lane'][index][0]
                laneInp[x][y][1] = scene['lane'][index][1]
                lane_norm_inp[x][y][0] = scene['lane_norm'][index][0]
                lane_norm_inp[x][y][1] = scene['lane_norm'][index][1]
            for y in range(30):
                index = closest_node(scene['p_out'][x][y],scene['lane'][:,:2])
                laneOut[x][y][0] = scene['lane'][index][0]
                laneOut[x][y][1] = scene['lane'][index][1]
                lane_norm_out[x][y][0] = scene['lane_norm'][index][0]
                lane_norm_out[x][y][1] = scene['lane_norm'][index][1]

        inpTotal.append(numpy.dstack([scene['p_in'],scene['v_in'],laneInp,lane_norm_inp]))
        outTotal.append(numpy.dstack([scene['p_out'], scene['v_out'],laneOut,lane_norm_out]))
#         inp.append(numpy.dstack([scene['p_in'],scene['v_in']]))
        out.append(numpy.dstack([scene['p_out'], scene['v_out']]))

#     inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    inpTotal = torch.FloatTensor(inpTotal)
    outTotal = torch.FloatTensor(outTotal)
    city = torch.FloatTensor(city)
    return [inpTotal,outTotal, out]

train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = True, collate_fn=my_collate, num_workers=0)

In [4]:

def val_collate(batch):
    agentIds = []
    trackIds = []
    sceneIdxs = []
    laneInfo = []
    inp = []
    city = []
    numbRows = 60
    for scene in batch:
        """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
        agentIds.append(scene['agent_id'])
        trackIds.append(scene['track_id'])
        sceneIdxs.append(scene['scene_idx'])
        
        cityyy = numpy.zeros((60,1,4))
        #print(cityyy)
        if scene['city'] == 'PIT':
            cityyy[:,:,:] = 1
        city.append(numpy.dstack([cityyy]))
        
        lanes = numpy.zeros((numbRows * 30,2))
        lane_norm = numpy.zeros((numbRows * 30,2))
        lengthLane = min(numbRows * 30,len(scene['lane']))
        lanes[:lengthLane,:2] = scene['lane'][:lengthLane,:2]
        lane_norm[:lengthLane,:2] = scene['lane_norm'] [:lengthLane,:2]
        laneInfo.append(numpy.dstack([lanes.reshape(60,30,2),lane_norm.reshape(60,30,2)]))
        inp.append(numpy.dstack([scene['p_in'], scene['v_in']]))
    inp = torch.FloatTensor(inp)
    laneInfo = torch.FloatTensor(laneInfo)
    city = torch.FloatTensor(city)
    return [inp,sceneIdxs,agentIds,trackIds,laneInfo,city]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=val_collate, num_workers=6)

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class RNNModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(RNNModel, self).__init__()
        

        self.hidden_dim = hidden_dim
        self.num_layers = n_layers
        
        self.lstm = nn.LSTM(input_size, self.hidden_dim, self.num_layers, batch_first=True,dropout=0.2)
        self.fc = nn.Conv1d(self.hidden_dim, 240,1)
#         self.bn1 = nn.BatchNorm1d(num_features=4)
        
    def forward(self, x):
        x,_ = self.lstm(x)
#         print("shapeee",x.shape)
        x = x.transpose(1,2)
#         print(x.shape)
        x = self.fc(x)
#         print(x.shape)
        x = x.transpose(1,2)

        return x
    
    def forwardTesting(self,x,num_steps=30):
        res =[]
        h = torch.zeros((self.num_layers,len(x),self.hidden_dim)).cuda()
        c = torch.zeros((self.num_layers,len(x),self.hidden_dim)).cuda()
        for steps in range(num_steps):
            x,(h,c) = self.lstm(x,(h,c))
            x = x[:,-1:]
            x = x.transpose(1,2)
            x = self.fc(x)
            x = x.transpose(1,2)
#             print("xxxx " ,x)
            res.append(x)
        res = torch.cat(res,1)
        return res


In [6]:
def train(i_batch,sample_batch,loss_ema):        
        inpTotal,outTotal, out = sample_batch
#             laneInfo = laneInfo.cuda()
        inpTotal = inpTotal.cuda()
        outTotal = outTotal.cuda()
#         city = city.cuda()
#         inp = inp.cuda()
        out = out.cuda()
        optimizer.zero_grad()
        mixed = torch.cat([inpTotal,outTotal],2).transpose(1,2).reshape(-1,49,8 * 60)
#         print(mixed.shape)
        y_pred = model(mixed[:,:-1])[:,-30:]
        y_pred = y_pred.reshape((-1,30,60,4)).transpose(1,2)

#         mixedOut = torch.cat([inp,out],2).transpose(1,2).reshape((-1,60,30,4))
#         print(y_pred.shape)
        loss = (torch.mean((y_pred-out)**2))
        loss.backward()
        optimizer.step()
        if loss_ema < 0:
            loss_ema = loss
        loss_ema= loss_ema*0.90 +loss*0.1

        if i_batch % 10 == 0:
            print("batch #: ",i_batch * batch_sz ," avg loss per scene(past 100): ",loss_ema.item(),loss.item())
            batch = []
        return loss_ema

### Visualize the batch of sequences

In [ ]:
from statistics import mean
import random
import numpy as np

torch.cuda.empty_cache()
agent_id = 0
learning_rate = 1e-2
momentum = 0.5
device = torch.device("cuda:0")
input_dim = 8 * 60    # input dimension
hidden_dim = 3000  # hidden layer dimension should be greater when batch_sz small
layer_dim = 3    # number of hidden layers
output_dim = 4   # output dimension
batch_sz = 200
n_epochs = 30

model = RNNModel(input_size=input_dim, output_size=output_dim, hidden_dim=hidden_dim, n_layers=layer_dim)
#model.load_state_dict(torch.load('./sliding3epoch.pth'))
model = model.to(device)
# optimizer = optim.Adagrad(model.parameters(), lr=learning_rate,lr_decay=1e-6,weight_decay=1e-5)
# optimizer = optim.Adadelta(model.parameters())
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-5,amsgrad=True)
#optimizer = optim.RMSprop(model.parameters(),lr=learning_rate,momentum=momentum)

#find mean
var = numpy.zeros((1,8))
seen = 0
print('start')

    
model.train()

print("test")
# newOut = torch.zeros((batch_sz,100,30,4))
batch = []
batches = []
loss_ema = -1
ranThroughWholeDataSet = False
try:
    for i_epoch in range(n_epochs):
        if ranThroughWholeDataSet == False:
            for i_batch, sample_batch in enumerate(train_loader):
#                 city,inpTotal,outTotal, out = sample_batch
#                 print(inpTotal[0])
                loss_ema = train(i_batch,sample_batch,loss_ema)
                batches.append(sample_batch)
            ranThroughWholeDataSet = True
        else:
            print("here")
            for i in range(len(batches)):
                loss_ema = train(i,batches[i],loss_ema) 
        
        torch.save(model.state_dict(), './sliding3epoch.pth')    
except KeyboardInterrupt:
    print("savedModel")
    torch.save(model.state_dict(), './sliding3epoch.pth')


start
test
batch #:  0  avg loss per scene(past 100):  358276.75 358276.75
batch #:  2000  avg loss per scene(past 100):  338526.375 326595.3125
batch #:  4000  avg loss per scene(past 100):  305296.3125 277710.21875
batch #:  6000  avg loss per scene(past 100):  277852.65625 238458.8125
batch #:  8000  avg loss per scene(past 100):  256201.3125 240327.484375
batch #:  10000  avg loss per scene(past 100):  234020.65625 216770.453125
batch #:  12000  avg loss per scene(past 100):  220312.125 235898.25


In [ ]:
torch.save(model.state_dict(), './sliding3epoch.pth')

In [ ]:
import numpy as np
# import pandas as pd

save_file = "submissionsliding1.csv"


header = ["ID"]
header += ["v"+str(x) for x in range(1, 61)]
print(header)

with open(save_file, 'w') as f:
    f.write(",".join(header)+"\n")

device = "cuda:0"

testmodel = RNNModel(input_size=input_dim, output_size=output_dim, hidden_dim=hidden_dim, n_layers=layer_dim)
testmodel.load_state_dict(torch.load('further3000Temp.pth'))
testmodel.to(device)
testmodel.eval()

full_out = []
print("test")
varMaxOutput= [4773.,   4097.7,   193.19,  194.33]
varMinOutput=[ -53.912,    0.,    -210.04,  -187.71 ]
#         #Changed to 4 because we don't need lane and lane norm
batch_sz = 1
     
for i_batch, sample_batch in enumerate(val_loader):
    if i_batch % 100 == 0:
        print("batch #: ", i_batch * batch_sz)

    inp, scene_idx, agent_ids, track_ids,laneInfo,city = sample_batch
    city = city.cuda()
    laneInfo = laneInfo.cuda()
    inp = inp.cuda()

    mixed = torch.cat([city,laneInfo,inp],2).transpose(1,2).reshape(-1,50,4 * 60)
    y_pred = testmodel.forwardTesting(mixed)
    y_pred = y_pred.reshape((-1,30,60,4)).transpose(1,2)

    i = 0
    out = []

    for ag_id in agent_ids:
        nestedTrackIds = track_ids[i]
        j = 0
        for track_id in nestedTrackIds:

            if ag_id == track_id[0]:
                out.append(y_pred[i][j][:,:2].reshape(60))
            j+=1
        i+=1
                           
 

    with open(save_file, "a") as f:
        for i in range(len(out)):
#             realOut = out[i].eval(session=tf.compat.v1.Session())
            array = out[i]
#             print(out[i][0])
            row = ""
            for element in array:
#                 print(element)
                row += str(element.item()) + ","
#             print("row ",row)
#         for row, scene_num in zip(realOut.astype(int).astype(str), scene_idx):
            f.write(str(scene_idx[i])+"," + row +"\n")
